In [1]:
import numpy as np

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
class LogicGate:
        
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        # 2:입력데이터 개수, 1: 출력데이터 개수, 4: 입력데이터(출력데이터) 2(1)개에 대한 조합(경우의수)==batch
        
        
        # 2층 hidden layer unit : 6개 가정,  가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,6)  # weight, 2 X 6 matrix, 입력데이터가 2개씩있음
        self.__b2 = np.random.rand(6) #노드의 수 6개를 갖는 벡터
        
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(6,1)
        self.__b3 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
    
        print(self.name + " object is created")
            
     # feed_foward & loss_val : 이전출력값이 다음입력값이 됨
    # 마지막 출력층에서 cross_entropy를 이용해 손실함수 값 계산

    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )    
    
    def loss_val(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
    
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
    
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
    
            if (step % 400 == 0):
                print("step = ", step, "  , loss value = ", self.loss_val())
                
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [3]:
# AND Gate 객체 생성 및 training 딥러닝 버전

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

AND object is created
Initial loss value =  5.73478406534876
step =  0   , loss value =  5.5294642626986406
step =  400   , loss value =  2.174758301841841
step =  800   , loss value =  1.9806802407160304
step =  1200   , loss value =  1.6944679574393797
step =  1600   , loss value =  1.3645302712817742
step =  2000   , loss value =  1.0464980167901756
step =  2400   , loss value =  0.779635398292675
step =  2800   , loss value =  0.5780829341964729
step =  3200   , loss value =  0.4348307276622819
step =  3600   , loss value =  0.3350761258500158
step =  4000   , loss value =  0.265174952726697
step =  4400   , loss value =  0.21521585072387783
step =  4800   , loss value =  0.17861900026848188
step =  5200   , loss value =  0.1511303284474566
step =  5600   , loss value =  0.1299918035980041
step =  6000   , loss value =  0.11338686650701088
step =  6400   , loss value =  0.10009370121655108
step =  6800   , loss value =  0.08927192663444543
step =  7200   , loss value =  0.080330671

In [6]:
#AND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(and_obj.predict(data))

(array([0.00015607]), 0)
(array([0.01176621]), 0)
(array([0.01160651]), 0)
(array([0.97832792]), 1)


In [8]:
# OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

OR object is created
Initial loss value =  3.2154906331625637
step =  0   , loss value =  3.1884414571467645
step =  400   , loss value =  1.9679011034278449
step =  800   , loss value =  1.7141888466173905
step =  1200   , loss value =  1.3256990215542184
step =  1600   , loss value =  0.907817674388384
step =  2000   , loss value =  0.5965224876562013
step =  2400   , loss value =  0.40313885455206644
step =  2800   , loss value =  0.28678229453213683
step =  3200   , loss value =  0.2145040472044796
step =  3600   , loss value =  0.16734893858941335
step =  4000   , loss value =  0.1350598492787781
step =  4400   , loss value =  0.111989981653699
step =  4800   , loss value =  0.09490263141006247
step =  5200   , loss value =  0.08185800035932767
step =  5600   , loss value =  0.07164328983241094
step =  6000   , loss value =  0.0634707392383659
step =  6400   , loss value =  0.056811100889791945
step =  6800   , loss value =  0.0512980966888325
step =  7200   , loss value =  0.0466

In [9]:
# OR Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(or_obj.predict(data))

(array([0.01558285]), 0)
(array([0.99408228]), 1)
(array([0.99389544]), 1)
(array([0.99987299]), 1)


In [10]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NAND object is created
Initial loss value =  3.234431243074649
step =  0   , loss value =  3.2046275029015328
step =  400   , loss value =  2.306431319738199
step =  800   , loss value =  2.1782720627367844
step =  1200   , loss value =  1.99542608418897
step =  1600   , loss value =  1.7171857993655606
step =  2000   , loss value =  1.385253885652841
step =  2400   , loss value =  1.0673253958611433
step =  2800   , loss value =  0.7989105815910953
step =  3200   , loss value =  0.593280046811395
step =  3600   , loss value =  0.445473456591291
step =  4000   , loss value =  0.34183562930896466
step =  4400   , loss value =  0.2689558799294831
step =  4800   , loss value =  0.21681785392829306
step =  5200   , loss value =  0.17867130546320442
step =  5600   , loss value =  0.15010813284349483
step =  6000   , loss value =  0.12824696987679424
step =  6400   , loss value =  0.11117595105925328
step =  6800   , loss value =  0.09760087461926492
step =  7200   , loss value =  0.08662723

In [11]:
# NAND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(nand_obj.predict(data))

(array([0.9998536]), 1)
(array([0.98783766]), 1)
(array([0.98774434]), 1)
(array([0.02145956]), 0)


In [12]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
Initial loss value =  5.649364011901657
step =  0   , loss value =  5.517268831661271
step =  400   , loss value =  2.7517510475131557
step =  800   , loss value =  2.744100927181152
step =  1200   , loss value =  2.734016341141256
step =  1600   , loss value =  2.7204987893065558
step =  2000   , loss value =  2.7023097595759307
step =  2400   , loss value =  2.6779816660362394
step =  2800   , loss value =  2.6458662781853493
step =  3200   , loss value =  2.604213880462217
step =  3600   , loss value =  2.551301873903154
step =  4000   , loss value =  2.4856673148146102
step =  4400   , loss value =  2.4064182236268525
step =  4800   , loss value =  2.3133435974274685
step =  5200   , loss value =  2.2064348832979146
step =  5600   , loss value =  2.08494998438211
step =  6000   , loss value =  1.9470229100171808
step =  6400   , loss value =  1.790972665388583
step =  6800   , loss value =  1.6180778162511222
step =  7200   , loss value =  1.4344760167860429
s

In [13]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.05378248]), 0)
(array([0.85787443]), 1)
(array([0.91882058]), 1)
(array([0.1657119]), 0)
